# Managing Device State

Recovering variables from previous notebooks:

In [1]:
#Hour and minute when this started to create unique names
%store -r unique
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A sample message payload
%store -r payload
#Thing name
%store -r thing_name

Create the MQTT Shadow client. This client will use a predefined topic structure to manage device state, but it is all plain old MQTT under the hood. See the [Using Shadows](https://docs.aws.amazon.com/iot/latest/developerguide/using-device-shadows.html) documentation page for more information.

In [2]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
client_id="ShadowClients{}".format(unique)
shadow = AWSIoTMQTTShadowClient(client_id)
shadow.configureEndpoint(endpoint_address, 8883)
shadow.configureCredentials(ca_file, private_key, certificate_file)
shadow.configureConnectDisconnectTimeout(600)
shadow.configureMQTTOperationTimeout(10)
shadow.connect()

True

On top of the client, a shadow handler is used to dispatch messages and callbacks:

In [3]:
shadow_handler = shadow.createShadowHandlerWithName(thing_name, True)

Let's report the initial device state. Suppose the device was just connected to power and should be standing by with the usual red led.

In [4]:
initial_state = {
    "power_status": "standby",
    "led_color": "red"
}

import json
shadow_state = {
    "state": {
        "reported" : initial_state
    }
}
shadow_str = json.dumps(shadow_state, indent = 4)
print(shadow_str)

{
    "state": {
        "reported": {
            "power_status": "standby",
            "led_color": "red"
        }
    }
}


In [5]:
def update_cb(payload, responseStatus, token):
    payload_json = json.loads(payload)
    print(json.dumps(payload_json,sort_keys=True,indent=4))
    print(str(responseStatus))
    print(str(token))

shadow_handler.shadowUpdate(shadow_str, update_cb, 5);

Notice that shadow messages include the metadata required for optimistic conflict resolution and other management information.

Confirm by retrieving the current shadow from AWS IoT:

In [6]:
def get_cb(payload, responseStatus, token):
    payload_json = json.loads(payload)
    print(json.dumps(payload_json,sort_keys=True,indent=4))
    print(str(responseStatus))
    print(str(token))

shadow_handler.shadowGet(get_cb, 5);

{
    "clientToken": "235df1a8-8ac0-44c6-9c61-6a73b8cc1b5a",
    "metadata": {
        "reported": {
            "led_color": {
                "timestamp": 1541068888
            },
            "power_status": {
                "timestamp": 1541068888
            }
        }
    },
    "state": {
        "reported": {
            "led_color": "red",
            "power_status": "standby"
        }
    },
    "timestamp": 1541068888,
    "version": 1
}
accepted
235df1a8-8ac0-44c6-9c61-6a73b8cc1b5a


In [7]:
#Wait for it
import time
time.sleep(1)

{
    "clientToken": "a9cfe8f8-1387-4389-a6e7-5f78c4c037e7",
    "metadata": {
        "reported": {
            "led_color": {
                "timestamp": 1541068888
            },
            "power_status": {
                "timestamp": 1541068888
            }
        }
    },
    "state": {
        "reported": {
            "led_color": "red",
            "power_status": "standby"
        }
    },
    "timestamp": 1541068891,
    "version": 1
}
accepted
a9cfe8f8-1387-4389-a6e7-5f78c4c037e7


Also try changing the device shadow of this thing using the AWS Console:

In [8]:
print("https://console.aws.amazon.com/iot/home?region=us-east-1#/thing/{}".format(thing_name))


https://console.aws.amazon.com/iot/home?region=us-east-1#/thing/bp_monitor_ehw104107


Now let's update the desired state. This will let the device know the desired configuration as soon as it is connected and ready to process deltas.

In [9]:
updated_state = {
    "led_color": "blue",
    "screen_brightness": "0.49"
}

shadow_state = {
    "state": {
        "desired" : updated_state
    }
}
shadow_str = json.dumps(shadow_state, indent=4)
shadow_handler.shadowUpdate(shadow_str, update_cb, 5);
print(shadow_str)

{
    "state": {
        "desired": {
            "led_color": "blue",
            "screen_brightness": "0.49"
        }
    }
}


On the device side, we can get the current shadow and either update the deltas or refresh to desired state:

In [10]:
shadow_handler.shadowGet(get_cb, 5);

With this data your application can always show the correct state to applications and execute the desired changes on the device without managing conectivity and databases.

Speaking of databases, we don't want to loose any data, right?

Let's proceed to the [Storing Telemetry Data](aws-iot-storing-telemetry.ipynb) notebook right now!

# Extra Credit

The MQTT Connection from the shadow client can be used to subscribe directly to the underlying message topics:

In [11]:
deltas_topic = "$aws/things/{}/shadow/update/delta".format(thing_name)
mqtt = shadow.getMQTTConnection()
# mqtt.subscribe...

{
    "clientToken": "52c70ddd-9767-489b-8d7e-81a630533c9e",
    "metadata": {
        "desired": {
            "led_color": {
                "timestamp": 1541068892
            },
            "screen_brightness": {
                "timestamp": 1541068892
            }
        }
    },
    "state": {
        "desired": {
            "led_color": "blue",
            "screen_brightness": "0.49"
        }
    },
    "timestamp": 1541068892,
    "version": 2
}
accepted
52c70ddd-9767-489b-8d7e-81a630533c9e
{
    "clientToken": "c4634b71-c404-4640-9eda-c1b2d8247ad5",
    "metadata": {
        "desired": {
            "led_color": {
                "timestamp": 1541068892
            },
            "screen_brightness": {
                "timestamp": 1541068892
            }
        },
        "reported": {
            "led_color": {
                "timestamp": 1541068888
            },
            "power_status": {
                "timestamp": 1541068888
            }
        }
    },
    "